# Homework 4
- 開始寫作業前，若使用 Colab 請先設定使用 GPU!!

In [1]:
# 0. 安裝套件

# !pip install torch==2.6.0 --index-url https://download.pytorch.org/whl/cu124
# !pip install -r requirements.txt

In [2]:
# 1. 載入套件

import torch
from pathlib import Path
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm import tqdm
from torchmetrics import SpearmanCorrCoef, Accuracy, F1Score

# Hugging Face PEFT
from peft import get_peft_model, LoraConfig, TaskType
from peft.utils.other import prepare_model_for_kbit_training

# 客製化模組
from dataset import SemevalDataset
from model import MultiLabelModel

/home/hjc/coSpace/NLP_HW4/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 2. 設定參數

MODEL_NAME = "microsoft/deberta-large" # "bert-base-uncased"
LR = 1e-5
NUM_EPOCHS = 3
TRAIN_BATCH_SIZE = 8
VAL_BATCH_SIZE = 8
SAVE_DIR = "./saved_models/"

# Create the directory if it doesn't exist
if not Path(SAVE_DIR).exists():
    Path(SAVE_DIR).mkdir(parents=True, exist_ok=False)

In [4]:
# 3. 載入資料集與測試

data_sample = SemevalDataset(split="train").data[:3]
print(f"Dataset example: \n{data_sample[0]} \n{data_sample[1]} \n{data_sample[2]}")

Dataset example: 
{'sentence_pair_id': 1, 'premise': 'A group of kids is playing in a yard and an old man is standing in the background', 'hypothesis': 'A group of boys in a yard is playing and a man is standing in the background', 'relatedness_score': 4.5, 'entailment_judgment': 0} 
{'sentence_pair_id': 2, 'premise': 'A group of children is playing in the house and there is no man standing in the background', 'hypothesis': 'A group of kids is playing in a yard and an old man is standing in the background', 'relatedness_score': 3.200000047683716, 'entailment_judgment': 0} 
{'sentence_pair_id': 3, 'premise': 'The young boys are playing outdoors and the man is smiling nearby', 'hypothesis': 'The kids are playing outdoors near a man with a smile', 'relatedness_score': 4.699999809265137, 'entailment_judgment': 1}


In [5]:
# 4. 載入 tokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir="./cache/")

In [6]:
# 5. 將 batch 資料進行整理
# 取出每筆資料的 'premise' 和 'hypothesis' 內容
# 將內容進行 tokenization 換成 token_ids 後，轉成 tensors
# 將 labels 也轉成 tensors

def collate_fn(batch):
    # TODO1: 完成 collate_fn
    # Write your code here
    # Extract premise and hypothesis from each example
    premises = [ex["premise"] for ex in batch]
    hypotheses = [ex["hypothesis"] for ex in batch]
    
    encoded = tokenizer(
        premises,
        hypotheses,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    # Prepare labels for regression (relatedness_score) and classification (entailment_judgment)
    scores = torch.tensor([ex["relatedness_score"] for ex in batch], dtype=torch.float)
    labels = torch.tensor([ex["entailment_judgment"] for ex in batch], dtype=torch.long)
    # Map to the expected variable names
    input_text = encoded
    labels1 = scores
    labels2 = labels
    return input_text, labels1, labels2

In [7]:
# 6. 建立 DataLoader

train_loader = DataLoader(
    SemevalDataset(split="train"),
    collate_fn=collate_fn,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
)
val_loader = DataLoader(
    SemevalDataset(split="validation"),
    collate_fn=collate_fn,
    batch_size=VAL_BATCH_SIZE,
    shuffle=False,
)

In [8]:
# 7. 設置 loss functions
# 因為是 multi-output learning
# 所以應該要有 2 種 loss functions

loss_fn1 = torch.nn.MSELoss()
loss_fn2 = torch.nn.CrossEntropyLoss()

In [9]:
# 8. 設置評估指標

spc = SpearmanCorrCoef()
acc = Accuracy(task="multiclass", num_classes=3)
f1 = F1Score(task="multiclass", num_classes=3, average='macro')

/home/hjc/coSpace/NLP_HW4/.venv/lib/python3.12/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [10]:
# 9. 載入模型，並直接把模型送至 GPU

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = MultiLabelModel(MODEL_NAME).to(device)

In [11]:
# 10. 配置LoRA
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=16,                          # LoRA矩陣的秩
    lora_alpha=32,                 # LoRA的縮放參數
    lora_dropout=0.1,              # LoRA層的dropout率
    bias="none",                   # 是否包含偏置參數
    target_modules=["query_proj", "key_proj", "value_proj", "output.dense"],  # 要應用LoRA的模塊
)

# 為主幹模型做準備
model = prepare_model_for_kbit_training(model)

# 將模型轉換為PEFT模型
model = get_peft_model(model, peft_config)

# 只訓練LoRA參數
for name, param in model.named_parameters():
    if "lora" not in name and "regression_head" not in name and "classification_head" not in name:
        param.requires_grad = False

# 印出可訓練參數的數量
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"可訓練參數數量: {trainable_params} ({trainable_params/total_params:.2%})")

可訓練參數數量: 2752512 (0.67%)


In [12]:
# 11. 載入模型與 optimizer

optimizer = AdamW(model.parameters(), lr = LR)

In [13]:
# 12. 建立測試函數

def do_test(
    dataloader,
    model,
    loss_fn1,
    loss_fn2,
    mode="validation",
    cur_epoch=0,
    num_epochs=NUM_EPOCHS,
):
    model.eval()
    
    pbar = tqdm(dataloader)
    pbar.set_description(f"{mode} epoch [{cur_epoch+1}/{NUM_EPOCHS}]")

    pred1 = torch.tensor([])
    pred2 = torch.tensor([])
    gt1 = torch.tensor([])
    gt2 = torch.tensor([])
    loss1 = 0
    loss2 = 0

    with torch.no_grad():
        for input_text, labels1, labels2 in pbar:
            outputs1, outputs2 = model(**input_text)

            loss1 += loss_fn1(outputs1, labels1).item()
            loss2 += loss_fn2(outputs2, labels2).item()

            outputs1 = outputs1.squeeze()
            outputs2 = torch.argmax(outputs2, dim=-1)
            pred1 = torch.cat((pred1, outputs1.to("cpu")), dim=-1)
            pred2 = torch.cat((pred2, outputs2.to("cpu")), dim=-1)
            gt1 = torch.cat((gt1, labels1.to("cpu")), dim=-1)
            gt2 = torch.cat((gt2, labels2.to("cpu")), dim=-1)

    print(f"Spearman Corr: {spc(pred1, gt1)} \nAccuracy: {acc(pred2, gt2)} \nF1 Score: {f1(pred2, gt2)}")
    loss1 /= len(dataloader)
    loss2 /= len(dataloader)
    return loss1, loss2

In [14]:
# Wrap validation loader 打包以避免一次將所有驗證集丟進GPU顯存，在我的本機上比較不會遇到OOM
class ValLoaderDeviceWrapper:
    def __init__(self, loader, device):
        self.loader = loader
        self.device = device
    def __iter__(self):
        for input_text, labels1, labels2 in self.loader:
            input_text = {k: v.to(self.device) for k, v in input_text.items()}
            labels1 = labels1.to(self.device)
            labels2 = labels2.to(self.device)
            yield input_text, labels1, labels2
    def __len__(self):
        return len(self.loader)

In [15]:
# Baseline (non-LoRA) 版本

baseline_model = MultiLabelModel(MODEL_NAME).to(device)
baseline_optimizer = AdamW(baseline_model.parameters(), lr=LR)
print("Starting baseline training (no LoRA)...")
for ep_base in range(NUM_EPOCHS):
    pbar_base = tqdm(train_loader)
    pbar_base.set_description(f"Baseline Training epoch [{ep_base+1}/{NUM_EPOCHS}]")
    baseline_model.train()
    for batch in pbar_base:
        baseline_optimizer.zero_grad()
        input_text_b, labels1_b, labels2_b = batch
        input_text_b = {k: v.to(device) for k, v in input_text_b.items()}
        labels1_b = labels1_b.to(device)
        labels2_b = labels2_b.to(device)
        score_b, logits_b = baseline_model(**input_text_b)
        loss1_b = loss_fn1(score_b, labels1_b)
        loss2_b = loss_fn2(logits_b, labels2_b)
        loss_b = loss1_b + loss2_b
        loss_b.backward()
        baseline_optimizer.step()
    val_loader_device = ValLoaderDeviceWrapper(val_loader, device)
    val_loss1_b, val_loss2_b = do_test(
        val_loader_device,
        baseline_model,
        loss_fn1,
        loss_fn2,
        mode="baseline_validation",
        cur_epoch=ep_base,
        num_epochs=NUM_EPOCHS,
    )

Starting baseline training (no LoRA)...


Baseline Training epoch [1/3]:  10%|▉         | 55/563 [00:12<01:52,  4.51it/s]


KeyboardInterrupt: 

In [ ]:
# LoRA 版本
for ep in range(NUM_EPOCHS):
    pbar = tqdm(train_loader)
    pbar.set_description(f"Training epoch [{ep+1}/{NUM_EPOCHS}]")
    model.train()
    for batch in pbar:
        # TODO3: Write the training loop
        # Write your code here
        # train your model
        # clear gradient
        optimizer.zero_grad()

        # Unpack tuple from collate_fn
        input_text, labels1, labels2 = batch

        # Move inputs and labels to device
        input_text = {k: v.to(device) for k, v in input_text.items()}
        labels1 = labels1.to(device)
        labels2 = labels2.to(device)

        # forward pass
        score_pred, logits_pred = model(**input_text)
        # compute loss
        loss1 = loss_fn1(score_pred, labels1)
        loss2 = loss_fn2(logits_pred, labels2)
        loss = loss1 + loss2
        # back-propagation
        loss.backward()
        # model optimization
        optimizer.step()

    # Create a generator that moves validation batches to the correct device
    def val_batches_on_device():
        for batch in val_loader:
            input_text, labels1, labels2 = batch
            input_text = {k: v.to(device) for k, v in input_text.items()}
            labels1 = labels1.to(device)
            labels2 = labels2.to(device)
            yield input_text, labels1, labels2

    val_loss1, val_loss2 = do_test(
        val_batches_on_device(),
        model,
        loss_fn1,
        loss_fn2,
        mode="validation",
        cur_epoch=ep,
        num_epochs=NUM_EPOCHS,
    )
    torch.save(model, f'./saved_models/ep{ep}.ckpt')
    print(f"Model saved to {SAVE_DIR}ep{ep}.ckpt!")